In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import gmaps.datasets
import os
import gmaps

# Import API key
from api_keys import g_key

#Imprort output from data acquisition and cleaning notebook
#clean_msp_home_sales_df = pd.DataFrame(pd.read_csv("clean_msp_tax_sales.csv"))
#clean_msp_home_sales_df.set_index('sale_id', inplace=True)
msp_community_summary_df = pd.DataFrame(pd.read_csv("msp_community_summary.csv"))

#Extract a short df for testing API's etc
#short_msp_home_sales_df = clean_msp_home_sales_df.iloc[0:100]

msp_community_summary_df

,community,House Size (sqft),Year Built,House Value,House Value per Sqft,LAT,LNG
0,CALHOUN-ISLE,2415.070730,1940.232503,525966.370107,217.785079,44.959319,-93.306924
1,CAMDEN,1931.847255,1936.081986,173586.012978,89.854937,45.032955,-93.300300
2,CENTRAL,1223.095072,1975.807787,381523.502781,311.932826,44.976510,-93.270469
3,LONGFELLOW,1949.829218,1928.355414,291723.015777,149.614650,44.948282,-93.227425
4,NEAR NORTH,2201.447637,1937.533322,177547.452814,80.650318,44.993191,-93.293128
5,NOKOMIS,2093.112378,1938.409419,304767.867889,145.605115,44.906686,-93.237680
6,NORTHEAST,2004.985241,1936.102749,262494.197837,130.920763,45.015793,-93.249438
7,PHILLIPS,1844.164526,1940.125994,184131.804281,99.845649,44.956520,-93.257054
8,POWDERHORN,1882.605671,1927.957364,233263.933089,123.904829,44.938932,-93.258480
9,SOUTHWEST,2457.285679,1934.550623,468208.916563,190.539065,44.916219,-93.298635


In [3]:
#Configure gmaps and read api key
gmaps.configure(api_key=g_key)

#Grab the needed columns for the humidity-weighted heatmap
home_price_heatmap = msp_community_summary_df[['LAT', 'LNG', 'House Value per Sqft']]

#Center on Minneapolis!
minneapolis = [44.971404, -93.266287]

#Set locations based on city lat long from WeatherPy
locations = home_price_heatmap[['LAT', 'LNG']]

#Weight the heatmap by humidity level
weights = home_price_heatmap['House Value per Sqft']

#Generate gmaps figure, add heatmap layer
fig = gmaps.figure(map_type='HYBRID', center = minneapolis, zoom_level=9.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

In [14]:
params = {
    "key": g_key,
}

for index, row in short_msp_home_sales_df.iterrows():
    # get address from df
    address = row["formatted_address"]
    address_new = address.replace("#", "%23")

    # change location each iteration while leaving original params in place
    params["address"] = f"{address_new}"
    
    #Base url and request
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        short_msp_home_sales_df.loc[index, 'Google Address'] = response['results'][0]['formatted_address']
        short_msp_home_sales_df.loc[index, 'Google Lat'] = response['results'][0]['geometry']['location']['lat']
        short_msp_home_sales_df.loc[index, 'Google Long'] = response['results'][0]['geometry']['location']['lng']
    except:
        print(f'Bad Google data for sale ID {index}')


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
#Configure gmaps and read api key
gmaps.configure(api_key=g_key)

#Grab the needed columns for the humidity-weighted heatmap
home_sale_price_heatmap = short_msp_home_sales_df[['Google Lat', 'Google Long', 'gross_sale_price']]

#Center on Minneapolis!
minneapolis = [44.971404, -93.266287]

#Set locations based on city lat long from WeatherPy
locations = home_sale_price_heatmap[['Google Lat', 'Google Long']]

#Weight the heatmap by humidity level
weights = home_sale_price_heatmap['gross_sale_price']

#Generate gmaps figure, add heatmap layer
fig = gmaps.figure(map_type='HYBRID', center = minneapolis, zoom_level=9.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
short_msp_home_sales_df.describe()

,community_cd,nbhd_cd,ward,adj_sale_price,gross_sale_price,downpayment,x_coord,y_coord,Google Lat,Google Long
count,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000
mean,5.590000,47.070000,6.730000,2.248260e+05,2.271056e+05,44509.680000,528856.642583,164286.356920,44.967384,-93.270885
std,3.143359,27.970495,3.868691,1.801465e+05,1.811649e+05,66073.697311,7860.377826,15715.532595,0.043099,0.030303
min,1.000000,1.000000,1.000000,1.200000e+04,1.200000e+04,0.000000,514292.092156,138157.215427,44.895673,-93.328169
25%,3.000000,25.750000,4.000000,1.195695e+05,1.195695e+05,690.000000,522486.232034,150679.431708,44.930289,-93.295298
50%,5.000000,44.500000,7.000000,1.765760e+05,1.808000e+05,16125.000000,528922.982430,163774.465160,44.965999,-93.270712
75%,9.000000,71.000000,10.000000,2.746250e+05,2.787500e+05,54925.000000,534321.729477,175494.680000,44.998112,-93.249640
max,11.000000,92.000000,13.000000,1.040000e+06,1.040000e+06,290000.000000,545306.952597,194773.766878,45.051005,-93.208629
